In [1]:
%AddDeps org.sameersingh.scalaplot scalaplot 0.0.4

Marking org.sameersingh.scalaplot:scalaplot:0.0.4 for download
Preparing to fetch from:
-> file:/tmp/toree_add_deps1258866547526972494/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree_add_deps1258866547526972494/https/repo1.maven.org/maven2/org/sameersingh/scalaplot/scalaplot/0.0.4/scalaplot-0.0.4.jar


In [2]:
val ss = org.apache.spark.sql.SparkSession.builder().getOrCreate()
import ss.implicits._
import org.apache.spark.sql.functions._

In [5]:
val df = Seq(("a a",2),("a a a a a",4)).toDF("a", "b")
df.withColumn("c", split($"a", " ")).show

+---------+---+---------------+
|        a|  b|              c|
+---------+---+---------------+
|      a a|  2|         [a, a]|
|a a a a a|  4|[a, a, a, a, a]|
+---------+---+---------------+



In [4]:
df.withColumn("c", length($"a")).show

+-----+---+---+
|    a|  b|  c|
+-----+---+---+
|   aa|  2|  2|
|aaaaa|  4|  5|
+-----+---+---+



### Preconditions
- Make sure data has been downloaded with `download_data.py`
- Make sure the first pass of cleaning has been performed with `clean_files.scala`

In [5]:
// val ss = org.apache.spark.sql.SparkSession.builder().getOrCreate()
// import ss.implicits._
// import org.apache.spark.sql.functions._

In [6]:


val files = List("cooking", "crypto", "robotics", "biology", "travel", "diy")

/* Load and print all files */
val df_all = files.map(f => {
                        ss.read.format("csv").option("header", "true").load("../dat/"+f+"_clean.csv")
                   }).reduce(_ union _).withColumn("tags", split($"tags", " "))

In [7]:
df_all.show

+---+--------------------+--------------------+--------------------+
| id|               title|             content|                tags|
+---+--------------------+--------------------+--------------------+
|  1|How can I get che...|<p>My chocolate c...|[baking, cookies,...|
|  2|How should I cook...|<p>I've heard of ...|[oven, cooking-ti...|
|  3|What is the diffe...|<p>I always use b...|              [eggs]|
|  4|What is the diffe...|<p>And can I use ...|[substitutions, p...|
|  5|In a tomato sauce...|<p>It seems that ...|[sauce, pasta, to...|
|  6|What ingredients ...|<p>I have a recip...|[substitutions, h...|
|  9|What is the inter...|<p>I'd like to kn...|[food-safety, bee...|
| 11|How should I poac...|<p>What's the bes...|[eggs, basics, po...|
| 12|How can I make my...|<p>My ice cream d...|         [ice-cream]|
| 17|How long and at w...|<p>I'm interested...|[baking, chicken,...|
| 23|Besides salmon, w...|<p>I've fallen in...|[grilling, salmon...|
| 27|Do I need to sift...|<p>Is th

In [11]:
df_all.count

87000

In [9]:
val rdd = df_all.select("tags").as[Seq[String]].rdd

In [10]:
val tags = rdd.collect.flatMap(_.map(t => (t, t.split("-").size)))
val tagsDF = tags.toSeq.distinct.toDF("tag", "length")

tags.maxBy(_._2)

(meet-in-the-middle-attack,5)

In [11]:
tagsDF.show

+--------------------+------+
|                 tag|length|
+--------------------+------+
|              baking|     1|
|             cookies|     1|
|             texture|     1|
|                oven|     1|
|        cooking-time|     2|
|               bacon|     1|
|                eggs|     1|
|       substitutions|     1|
|please-remove-thi...|     4|
|         baking-soda|     2|
|       baking-powder|     2|
|               sauce|     1|
|               pasta|     1|
|            tomatoes|     1|
|     italian-cuisine|     2|
|               herbs|     1|
|             parsley|     1|
|         food-safety|     2|
|                beef|     1|
|              basics|     1|
+--------------------+------+
only showing top 20 rows



### TODO: load all data and do a histogram over tag lengths

In [12]:
tagsDF.sort(-$"length").show(false)

+-------------------------+------+
|tag                      |length|
+-------------------------+------+
|meet-in-the-middle-attack|5     |
|ho-chi-minh-city         |4     |
|change-purpose-of-travel |4     |
|san-francisco-bay-area   |4     |
|please-remove-this-tag   |4     |
|us-visa-waiver-program   |4     |
|man-in-the-middle        |4     |
|proof-provenance-of-funds|4     |
|t-and-t-citizens         |4     |
|90-180-visa-rules        |4     |
|great-wall-of-china      |4     |
|proof-of-onward-travel   |4     |
|gondolas-and-cable-cars  |4     |
|self-leveling-concrete   |3     |
|glass-top-range          |3     |
|diy-vs-pro               |3     |
|damp-proof-course        |3     |
|thermostat-c-wire        |3     |
|grounding-and-bonding    |3     |
|sliding-glass-door       |3     |
+-------------------------+------+
only showing top 20 rows



In [13]:
import org.sameersingh.scalaplot.Implicits._
val data = tags.map(_._2).groupBy(identity).mapValues(_.length)
val (x, y) = data.toSeq.sortBy(_._1).map(x => (x._1.toDouble, x._2.toDouble)).unzip

In [14]:
kernel.magics.html(output(SVG, barChart(x -> y)))

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.0 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 20000 
 
 
 
 
 40000 
 
 
 
 
 60000 
 
 
 
 
 80000 
 
 
 
 
 100000 
 
 
 
 
 120000 
 
 
 
 
 140000 
 
 
 
 
 160000 
 
 
 
 
 180000 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Label 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Label

In [15]:
val tagMatcher = """<((?!>).{1,5})>""".r
val htmlTags = df_all.select("content").as[String].flatMap(x => tagMatcher.findAllIn(x).toList).distinct.collect

Name: org.apache.spark.SparkException
Message: Task not serializable
StackTrace:   at org.apache.spark.util.ClosureCleaner$.ensureSerializable(ClosureCleaner.scala:298)
  at org.apache.spark.util.ClosureCleaner$.org$apache$spark$util$ClosureCleaner$$clean(ClosureCleaner.scala:288)
  at org.apache.spark.util.ClosureCleaner$.clean(ClosureCleaner.scala:108)
  at org.apache.spark.SparkContext.clean(SparkContext.scala:2056)
  at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsWithIndex$1.apply(RDD.scala:817)
  at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsWithIndex$1.apply(RDD.scala:816)
  at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
  at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
  at org.apache.spark.rdd.RDD.withScope(RDD.scala:358)
  at org.apache.spark.rdd.RDD.mapPartitionsWithIndex(RDD.scala:816)
  at org.apache.spark.sql.execution.WholeStageCodegenExec.doExecute(WholeStageCodegenExec.scala:364)
  at org.apache

In [16]:
println(s"Number of tags: ${htmlTags.length}")
htmlTags.foreach{println}

Name: Unknown Error
Message: lastException: Throwable = null
<console>:32: error: not found: value htmlTags
       println(s"Number of tags: ${htmlTags.length}")
                                   ^
StackTrace: 